In [78]:
%load_ext autoreload
%autoreload 2


from util_0630 import *
from myalgorithm_0630_1 import algorithm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- 실험해볼 내용들

1. 가까운 거리는 먼저 WALK를 배정하면 어떨까?
2. 배달원을 전부 한번에 재할당해보면 어떨까?
3. 가까운 번들 내의 원소를 교환하는 휴리스틱 코드를 만들면 어떨까?

In [263]:
problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'

### 초기 크루스칼 번들링

In [268]:
## Basic Code
# problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

car_rider = ALL_RIDERS[2]

init_availables = [rider.available_number for rider in ALL_RIDERS]

## Custom Code
def get_init_bundle(weight1, weight2):
    for rider_i in range(3):
        ALL_RIDERS[rider_i].available_number = init_availables[rider_i]

    def find(v):
        while v != parent[v]:
            parent[v] = parent[parent[v]]
            v = parent[v]

        return v

    def union(a, b, new_bundle):
        if a > b:
            a, b = b, a

        parent[b] = a
        all_bundles[a] = new_bundle

    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    car_rider = ALL_RIDERS[2]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    for i in range(len(all_bundles)):
        bundle = all_bundles[i]

        shop_seq = bundle.shop_seq

        xs_s_sum = 0
        ys_s_sum = 0

        xs_e_sum = 0
        ys_e_sum = 0

        readytimes_sum = 0
        deadlines_sum = 0

        shop_seq_len = len(shop_seq)

        for order_num in shop_seq:
            order = ALL_ORDERS[order_num]

            xs_s_sum += order.shop_lat
            ys_s_sum += order.shop_lon

            xs_e_sum += order.dlv_lat
            ys_e_sum += order.dlv_lon

            readytimes_sum += order.ready_time
            deadlines_sum += order.deadline

        xs_s_avg = xs_s_sum / shop_seq_len
        ys_s_avg = ys_s_sum / shop_seq_len

        xs_e_avg = xs_e_sum / shop_seq_len
        ys_e_avg = ys_e_sum / shop_seq_len

        readytimes_avg = readytimes_sum / shop_seq_len
        deadlines_avg = deadlines_sum / shop_seq_len

        avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

        bundle.avg_info = avg_info

    edges = []
    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            avg_info1 = all_bundles[i].avg_info
            avg_info2 = all_bundles[j].avg_info

            sx1, sy1, ex1, ey1, r1, d1 = avg_info1
            sx2, sy2, ex2, ey2, r2, d2 = avg_info2

            r_diff = abs(r1 - r2)
            d_diff = abs(d1 - d2)

            start_end_diff = get_dist_by_coords((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

            dist1 = DIST[i][j]
            dist2 = DIST[i + K][j + K]

            diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

            edges.append((i, j, diff_score))

    parent = list(range(K))
    edges.sort(key=lambda x: x[2])

    for bundle_num1, bundle_num2, diff_score in edges:
        rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

        if rbn1 == rbn2:
            continue
        
        new_bundle = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2])
        if new_bundle is not None:
            all_bundles[rbn1].rider.available_number += 1
            all_bundles[rbn2].rider.available_number += 1
            new_bundle.rider.available_number -= 1

            union(rbn1, rbn2, new_bundle)

    parent = [find(v) for v in parent]

    result_bundles = [all_bundles[v] for v in set(parent)]
    result_availables = [rider.available_number for rider in ALL_RIDERS]

    return result_bundles, result_availables, sum((bundle.cost for bundle in result_bundles)) / K

min_init_cost = 100000000000000000
min_init_cost_bundle = []
min_init_cost_rider_availables = []
for weight1 in [0, 1]:
    for weight2 in [-3, -2, -1.5, -1, -0.5, 0, 0.5]:
        bundles, result_rider_availables, cost = get_init_bundle(weight1, weight2)

        if cost < min_init_cost:
            min_init_cost = cost
            min_init_cost_bundles = bundles
            min_init_cost_rider_availables = result_rider_availables

for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = min_init_cost_rider_availables[rider_i]
all_bundles = min_init_cost_bundles

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

## 결과 실현 가능성 확인

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])


4187.728333333333


### 초기 크루스칼 번들링 및 배달원 재할당 케이스

In [282]:
## Basic Code
# problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

car_rider = ALL_RIDERS[2]

init_availables = [rider.available_number for rider in ALL_RIDERS]

## Custom Code
def get_init_bundle(weight1, weight2):
    for rider_i in range(3):
        ALL_RIDERS[rider_i].available_number = init_availables[rider_i]

    def find(v):
        while v != parent[v]:
            parent[v] = parent[parent[v]]
            v = parent[v]

        return v

    def union(a, b, new_bundle):
        if a > b:
            a, b = b, a

        parent[b] = a
        all_bundles[a] = new_bundle

    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    car_rider = ALL_RIDERS[2]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    for i in range(len(all_bundles)):
        bundle = all_bundles[i]

        shop_seq = bundle.shop_seq

        xs_s_sum = 0
        ys_s_sum = 0

        xs_e_sum = 0
        ys_e_sum = 0

        readytimes_sum = 0
        deadlines_sum = 0

        shop_seq_len = len(shop_seq)

        for order_num in shop_seq:
            order = ALL_ORDERS[order_num]

            xs_s_sum += order.shop_lat
            ys_s_sum += order.shop_lon

            xs_e_sum += order.dlv_lat
            ys_e_sum += order.dlv_lon

            readytimes_sum += order.ready_time
            deadlines_sum += order.deadline

        xs_s_avg = xs_s_sum / shop_seq_len
        ys_s_avg = ys_s_sum / shop_seq_len

        xs_e_avg = xs_e_sum / shop_seq_len
        ys_e_avg = ys_e_sum / shop_seq_len

        readytimes_avg = readytimes_sum / shop_seq_len
        deadlines_avg = deadlines_sum / shop_seq_len

        avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

        bundle.avg_info = avg_info

    edges = []
    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            avg_info1 = all_bundles[i].avg_info
            avg_info2 = all_bundles[j].avg_info

            sx1, sy1, ex1, ey1, r1, d1 = avg_info1
            sx2, sy2, ex2, ey2, r2, d2 = avg_info2

            r_diff = abs(r1 - r2)
            d_diff = abs(d1 - d2)

            start_end_diff = get_dist_by_coords((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

            dist1 = DIST[i][j]
            dist2 = DIST[i + K][j + K]

            diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

            edges.append((i, j, diff_score))

    parent = list(range(K))
    edges.sort(key=lambda x: x[2])

    for bundle_num1, bundle_num2, diff_score in edges:
        rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

        if rbn1 == rbn2:
            continue
        
        new_bundle = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2])
        if new_bundle is not None:
            all_bundles[rbn1].rider.available_number += 1
            all_bundles[rbn2].rider.available_number += 1
            new_bundle.rider.available_number -= 1

            union(rbn1, rbn2, new_bundle)

    parent = [find(v) for v in parent]

    result_bundles = [all_bundles[v] for v in set(parent)]
    result_availables = [rider.available_number for rider in ALL_RIDERS]

    return result_bundles, result_availables, sum((bundle.cost for bundle in result_bundles)) / K

min_init_cost = 100000000000000000
min_init_cost_bundle = []
min_init_cost_rider_availables = []
for weight1 in [0, 1]:
    for weight2 in [-3, -2, -1.5, -1, -0.5, 0, 0.5]:
        bundles, result_rider_availables, cost = get_init_bundle(weight1, weight2)

        if cost < min_init_cost:
            min_init_cost = cost
            min_init_cost_bundles = bundles
            min_init_cost_rider_availables = result_rider_availables

for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = min_init_cost_rider_availables[rider_i]
all_bundles = min_init_cost_bundles

for bundle in all_bundles:
    new_rider = get_cheaper_available_riders(ALL_RIDERS, bundle.rider)
    if new_rider is not None:
        old_rider = bundle.rider

        check_result = check_bundle_rider_changing(ALL_ORDERS, bundle, new_rider)
        if check_result:
            bundle.shop_seq = check_result[0]
            bundle.dlv_seq = check_result[1]
            bundle.rider = check_result[2]
            bundle.update_cost()

            old_rider.available_number += 1
            new_rider.available_number -= 1

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

## 결과 실현 가능성 확인

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])


4084.696666666666


### try_merging_bundles_by_cost를 사용하여 번들 합칠 때 비용을 보는 케이스

In [296]:
## Basic Code
# problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

car_rider = ALL_RIDERS[2]

init_availables = [rider.available_number for rider in ALL_RIDERS]

## Custom Code
def get_init_bundle(weight1, weight2):
    for rider_i in range(3):
        ALL_RIDERS[rider_i].available_number = init_availables[rider_i]

    def find(v):
        while v != parent[v]:
            parent[v] = parent[parent[v]]
            v = parent[v]

        return v

    def union(a, b, new_bundle):
        if a > b:
            a, b = b, a

        parent[b] = a
        all_bundles[a] = new_bundle

    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    car_rider = ALL_RIDERS[2]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    for i in range(len(all_bundles)):
        bundle = all_bundles[i]

        shop_seq = bundle.shop_seq

        xs_s_sum = 0
        ys_s_sum = 0

        xs_e_sum = 0
        ys_e_sum = 0

        readytimes_sum = 0
        deadlines_sum = 0

        shop_seq_len = len(shop_seq)

        for order_num in shop_seq:
            order = ALL_ORDERS[order_num]

            xs_s_sum += order.shop_lat
            ys_s_sum += order.shop_lon

            xs_e_sum += order.dlv_lat
            ys_e_sum += order.dlv_lon

            readytimes_sum += order.ready_time
            deadlines_sum += order.deadline

        xs_s_avg = xs_s_sum / shop_seq_len
        ys_s_avg = ys_s_sum / shop_seq_len

        xs_e_avg = xs_e_sum / shop_seq_len
        ys_e_avg = ys_e_sum / shop_seq_len

        readytimes_avg = readytimes_sum / shop_seq_len
        deadlines_avg = deadlines_sum / shop_seq_len

        avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

        bundle.avg_info = avg_info

    edges = []
    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            avg_info1 = all_bundles[i].avg_info
            avg_info2 = all_bundles[j].avg_info

            sx1, sy1, ex1, ey1, r1, d1 = avg_info1
            sx2, sy2, ex2, ey2, r2, d2 = avg_info2

            r_diff = abs(r1 - r2)
            d_diff = abs(d1 - d2)

            start_end_diff = get_dist_by_coords((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

            dist1 = DIST[i][j]
            dist2 = DIST[i + K][j + K]

            diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

            edges.append((i, j, diff_score))

    parent = list(range(K))
    edges.sort(key=lambda x: x[2])

    for bundle_num1, bundle_num2, diff_score in edges:
        rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

        if rbn1 == rbn2:
            continue

        
        
        new_bundle = try_merging_bundles_by_cost(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2])
        if new_bundle is not None:
            all_bundles[rbn1].rider.available_number += 1
            all_bundles[rbn2].rider.available_number += 1
            new_bundle.rider.available_number -= 1

            union(rbn1, rbn2, new_bundle)

    parent = [find(v) for v in parent]

    result_bundles = [all_bundles[v] for v in set(parent)]
    result_availables = [rider.available_number for rider in ALL_RIDERS]

    return result_bundles, result_availables, sum((bundle.cost for bundle in result_bundles)) / K

min_init_cost = 100000000000000000
min_init_cost_bundle = []
min_init_cost_rider_availables = []
for weight1 in [0, 1]:
    for weight2 in [-3, -2, -1.5, -1, -0.5, 0, 0.5]:
        bundles, result_rider_availables, cost = get_init_bundle(weight1, weight2)

        if cost < min_init_cost:
            min_init_cost = cost
            min_init_cost_bundles = bundles
            min_init_cost_rider_availables = result_rider_availables

for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = min_init_cost_rider_availables[rider_i]
all_bundles = min_init_cost_bundles

for bundle in all_bundles:
    new_rider = get_cheaper_available_riders(ALL_RIDERS, bundle.rider)
    if new_rider is not None:
        old_rider = bundle.rider

        check_result = check_bundle_rider_changing(ALL_ORDERS, bundle, new_rider)
        if check_result:
            bundle.shop_seq = check_result[0]
            bundle.dlv_seq = check_result[1]
            bundle.rider = check_result[2]
            bundle.update_cost()

            old_rider.available_number += 1
            new_rider.available_number -= 1

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

## 결과 실현 가능성 확인

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])


4158.910333333333


> 비용을 기준으로 했을 때 결과가 오히려 안좋음

### 우선적으로 도보 배달원으로 할당하는 케이스

In [311]:
## Basic Code
# problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

car_rider = ALL_RIDERS[2]

init_availables = [rider.available_number for rider in ALL_RIDERS]

## Custom Code
def get_init_bundle(weight1, weight2):
    for rider_i in range(3):
        ALL_RIDERS[rider_i].available_number = init_availables[rider_i]

    def find(v):
        while v != parent[v]:
            parent[v] = parent[parent[v]]
            v = parent[v]

        return v

    def union(a, b, new_bundle):
        if a > b:
            a, b = b, a

        parent[b] = a
        all_bundles[a] = new_bundle

    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    car_rider = ALL_RIDERS[2]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    for i in range(len(all_bundles)):
        bundle = all_bundles[i]

        shop_seq = bundle.shop_seq

        xs_s_sum = 0
        ys_s_sum = 0

        xs_e_sum = 0
        ys_e_sum = 0

        readytimes_sum = 0
        deadlines_sum = 0

        shop_seq_len = len(shop_seq)

        for order_num in shop_seq:
            order = ALL_ORDERS[order_num]

            xs_s_sum += order.shop_lat
            ys_s_sum += order.shop_lon

            xs_e_sum += order.dlv_lat
            ys_e_sum += order.dlv_lon

            readytimes_sum += order.ready_time
            deadlines_sum += order.deadline

        xs_s_avg = xs_s_sum / shop_seq_len
        ys_s_avg = ys_s_sum / shop_seq_len

        xs_e_avg = xs_e_sum / shop_seq_len
        ys_e_avg = ys_e_sum / shop_seq_len

        readytimes_avg = readytimes_sum / shop_seq_len
        deadlines_avg = deadlines_sum / shop_seq_len

        avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

        bundle.avg_info = avg_info

    edges = []
    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            avg_info1 = all_bundles[i].avg_info
            avg_info2 = all_bundles[j].avg_info

            sx1, sy1, ex1, ey1, r1, d1 = avg_info1
            sx2, sy2, ex2, ey2, r2, d2 = avg_info2

            r_diff = abs(r1 - r2)
            d_diff = abs(d1 - d2)

            start_end_diff = get_dist_by_coords((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

            dist1 = DIST[i][j]
            dist2 = DIST[i + K][j + K]

            diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

            edges.append((i, j, diff_score))

    parent = list(range(K))
    edges.sort(key=lambda x: x[2])

    for bundle_num1, bundle_num2, diff_score in edges:
        rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

        if rbn1 == rbn2:
            continue
        
        new_bundle = try_merging_bundles_by_dist_walk_prefered(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2])
        if new_bundle is not None:
            all_bundles[rbn1].rider.available_number += 1
            all_bundles[rbn2].rider.available_number += 1
            new_bundle.rider.available_number -= 1

            union(rbn1, rbn2, new_bundle)

    parent = [find(v) for v in parent]

    result_bundles = [all_bundles[v] for v in set(parent)]
    result_availables = [rider.available_number for rider in ALL_RIDERS]

    return result_bundles, result_availables, sum((bundle.cost for bundle in result_bundles)) / K

min_init_cost = 100000000000000000
min_init_cost_bundle = []
min_init_cost_rider_availables = []
for weight1 in [0, 1]:
    for weight2 in [-3, -2, -1.5, -1, -0.5, 0, 0.5]:
        bundles, result_rider_availables, cost = get_init_bundle(weight1, weight2)

        if cost < min_init_cost:
            min_init_cost = cost
            min_init_cost_bundles = bundles
            min_init_cost_rider_availables = result_rider_availables

for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = min_init_cost_rider_availables[rider_i]
all_bundles = min_init_cost_bundles

for bundle in all_bundles:
    new_rider = get_cheaper_available_riders(ALL_RIDERS, bundle.rider)
    if new_rider is not None:
        old_rider = bundle.rider

        check_result = check_bundle_rider_changing(ALL_ORDERS, bundle, new_rider)
        if check_result:
            bundle.shop_seq = check_result[0]
            bundle.dlv_seq = check_result[1]
            bundle.rider = check_result[2]
            bundle.update_cost()

            old_rider.available_number += 1
            new_rider.available_number -= 1

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

## 결과 실현 가능성 확인

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])


4063.125999999998


### 배달원 재할당 코드 제작

In [312]:
[rider.type for rider in ALL_RIDERS]

['BIKE', 'WALK', 'CAR']

In [313]:
inf = float('inf')

rider_types = [rider.type for rider in ALL_RIDERS]
car_rider_index = rider_types.index('CAR')
walk_rider_index = rider_types.index('WALK')
bike_rider_index = rider_types.index('BIKE')

rider_availables = init_availables.copy()

bundle_info = [[''] * 3 for _ in range(len(all_bundles))]
cost_info = [[inf] * 3 for _ in range(len(all_bundles))]

for rider_i in range(3):
    rider = ALL_RIDERS[rider_i]

    for bundle_i in range(len(all_bundles)):
        old_bundle = all_bundles[bundle_i]
        volume = old_bundle.total_volume

        checked = check_bundle_rider_changing2(ALL_ORDERS, all_bundles[bundle_i], rider)

        if checked:
            shop_seq, dlv_seq, _ = checked

            new_bundle = Bundle(ALL_ORDERS, rider, shop_seq, dlv_seq, volume, get_total_distance(K, DIST, shop_seq, dlv_seq))

            bundle_info[bundle_i][rider_i] = new_bundle
            cost_info[bundle_i][rider_i] = new_bundle.cost

In [314]:
print(len(bundle_info))
for i in range(len(bundle_info)):
    print(i, f'{cost_info[i][0]}, {cost_info[i][1]}, {cost_info[i][2]}')

112
0 12468.0, inf, inf
1 13559.4, inf, inf
2 10639.8, 4644.1, 7733.0
3 inf, inf, 11232.0
4 13518.0, inf, 13869.0
5 14559.599999999999, inf, 15956.0
6 14517.599999999999, inf, 15283.0
7 14452.2, inf, 11924.0
8 11532.0, inf, 9511.0
9 11149.8, inf, 10102.0
10 inf, inf, 11557.0
11 10982.4, 4718.2, 8304.0
12 12063.0, inf, 11709.0
13 14135.400000000001, inf, 13011.0
14 11741.4, inf, 9591.0
15 12631.8, inf, 12115.0
16 11495.4, inf, 9159.0
17 12368.4, inf, 11423.0
18 14965.8, inf, 11858.0
19 18981.0, inf, inf
20 11341.8, inf, 9707.0
21 13105.2, inf, 11842.0
22 15831.6, inf, 16712.0
23 10334.4, 4667.2, 7224.0
24 10709.4, 4854.7, 7849.0
25 14838.6, inf, inf
26 12304.2, inf, 12010.0
27 11625.0, inf, 8762.0
28 10869.0, 4693.0, 8115.0
29 11398.8, inf, 8808.0
30 16593.0, inf, inf
31 12442.8, inf, 10738.0
32 10453.8, 4726.9, 7423.0
33 12349.2, inf, 10150.0
34 10954.2, 4977.1, 8257.0
35 12807.0, inf, 12083.0
36 11374.2, inf, 8957.0
37 11416.8, inf, 8279.0
38 10451.4, 4574.8, 7419.0
39 12631.2, inf, 1

In [315]:
# 한 종류의 라이더밖에 할당이 안되는 경우 또는 CAR이 가장 비용이 낮은 경우에 우선 할당
to_dels = []
new_all_bundles = []
for i in range(len(bundle_info)):
    cost0, cost1, cost2 = cost_info[i]

    if cost0 == cost1 == inf:
        new_all_bundles.append(bundle_info[i][2])
        rider_availables[2] -= 1
        to_dels.append(i)
    elif cost0 == cost2 == inf:
        new_all_bundles.append(bundle_info[i][1])
        rider_availables[1] -= 1
        to_dels.append(i)       
    elif cost1 == cost2 == inf:
        new_all_bundles.append(bundle_info[i][0])
        rider_availables[0] -= 1
        to_dels.append(i)
    elif cost_info[i][car_rider_index] == min(cost_info[i]):
        new_all_bundles.append(bundle_info[i][car_rider_index])
        rider_availables[car_rider_index] -= 1
        to_dels.append(i)       

to_dels = set(to_dels)
bundle_info = [bundle_info[i] for i in range(len(bundle_info)) if i not in to_dels]
cost_info = [cost_info[i] for i in range(len(cost_info)) if i not in to_dels]

In [316]:
print(len(bundle_info))
for i in range(len(bundle_info)):
    print(i, f'{cost_info[i][0]}, {cost_info[i][1]}, {cost_info[i][2]}')

37
0 10639.8, 4644.1, 7733.0
1 13518.0, inf, 13869.0
2 14559.599999999999, inf, 15956.0
3 14517.599999999999, inf, 15283.0
4 10982.4, 4718.2, 8304.0
5 15831.6, inf, 16712.0
6 10334.4, 4667.2, 7224.0
7 10709.4, 4854.7, 7849.0
8 10869.0, 4693.0, 8115.0
9 10453.8, 4726.9, 7423.0
10 10954.2, 4977.1, 8257.0
11 10451.4, 4574.8, 7419.0
12 9317.4, 4158.7, 5529.0
13 12996.6, inf, 13154.0
14 13270.2, inf, 13282.0
15 13161.6, inf, 13945.0
16 14658.6, inf, 15274.0
17 13383.6, inf, 13664.0
18 15991.2, inf, 16652.0
19 12873.6, inf, 14021.0
20 14929.8, inf, 17753.0
21 10325.4, 4662.7, 7209.0
22 15629.4, inf, 16049.0
23 12772.2, inf, 13168.0
24 13048.8, inf, 14864.0
25 12989.4, inf, 15055.0
26 10899.6, 4949.8, 8166.0
27 11926.2, 5156.8, 8856.0
28 10362.0, 4562.2, 7270.0
29 15946.2, inf, 17172.0
30 9778.8, 4389.4, 6298.0
31 12133.2, 5566.6, 10222.0
32 9819.0, 4409.5, 6365.0
33 11266.8, 5133.4, 8778.0
34 10081.2, 4540.6, 6802.0
35 10410.6, 4705.3, 7351.0
36 9142.8, 4071.4, 5238.0


In [317]:
rider_availables

[52, 90, 233]

In [318]:
# WALK가 비용이 가장 낮은 경우 우선 할당 - 만약 WALK 배달원이 부족한 경우 두 번째로 비용이 적게 드는 경우와의 차이가 가장 큰 번들부터 할당
walk_cands = []
for i in range(len(bundle_info)):
    cost_info_sorted = sorted(cost_info[i])
    if cost_info[i][walk_rider_index] == min(cost_info[i]):
        diff = cost_info_sorted[1] - cost_info[i][walk_rider_index]
        walk_cands.append((diff, i))

walk_cands.sort(key=lambda x: -x[0])
walk_rider_available = rider_availables[walk_rider_index]
walk_cands = walk_cands[:walk_rider_available]

walk_cands_indices = []
for _, i in walk_cands:
    walk_cands_indices.append(i)

    new_all_bundles.append(bundle_info[i][walk_rider_index])
    rider_availables[walk_rider_index] -= 1  

walk_cands_indices = set(walk_cands_indices)
bundle_info = [bundle_info[i] for i in range(len(bundle_info)) if i not in walk_cands_indices]
cost_info = [cost_info[i] for i in range(len(cost_info)) if i not in walk_cands_indices]

In [319]:
print(len(bundle_info))
for i in range(len(bundle_info)):
    print(i, f'{cost_info[i][0]}, {cost_info[i][1]}, {cost_info[i][2]}')

17
0 13518.0, inf, 13869.0
1 14559.599999999999, inf, 15956.0
2 14517.599999999999, inf, 15283.0
3 15831.6, inf, 16712.0
4 12996.6, inf, 13154.0
5 13270.2, inf, 13282.0
6 13161.6, inf, 13945.0
7 14658.6, inf, 15274.0
8 13383.6, inf, 13664.0
9 15991.2, inf, 16652.0
10 12873.6, inf, 14021.0
11 14929.8, inf, 17753.0
12 15629.4, inf, 16049.0
13 12772.2, inf, 13168.0
14 13048.8, inf, 14864.0
15 12989.4, inf, 15055.0
16 15946.2, inf, 17172.0


In [320]:
# BIKE가 비용이 가장 낮은 경우 우선 할당 - 만약 BIKE 배달원이 부족한 경우 CAR 배달원하고의 비용 차이가 가장 큰 번들부터 할당
bike_cands = []
for i in range(len(bundle_info)):
    cost_info_sorted = sorted(cost_info[i])
    if cost_info[i][bike_rider_index] == min(cost_info[i]):
        diff = cost_info[i][car_rider_index] - cost_info[i][bike_rider_index]
        bike_cands.append((diff, i))

bike_cands.sort(key=lambda x: -x[0])
bike_rider_available = rider_availables[bike_rider_index]
bike_cands = bike_cands[:bike_rider_available]

bike_cands_indices = []
for _, i in bike_cands:
    bike_cands_indices.append(i)

    new_all_bundles.append(bundle_info[i][bike_rider_index])
    rider_availables[bike_rider_index] -= 1  

bike_cands_indices = set(bike_cands_indices)
bundle_info = [bundle_info[i] for i in range(len(bundle_info)) if i not in bike_cands_indices]
cost_info = [cost_info[i] for i in range(len(cost_info)) if i not in bike_cands_indices]

In [321]:
print(len(bundle_info))
for i in range(len(bundle_info)):
    print(i, f'{cost_info[i][0]}, {cost_info[i][1]}, {cost_info[i][2]}')

0


In [322]:
# 남은 배달원 중 가장 적은 비용을 가진 배달원에 바로 할당
to_dels = []
for i in range(len(bundle_info)):
    costs = [(cost_info[i][j], rider_types[j]) for j in range(3)]
    costs.sort(key=lambda x: x[0])

    for cost, rider_type in costs:
        rider_index = rider_types.index(rider_type)
        if rider_availables[rider_index]:
            rider_availables[rider_index] -= 1
            new_all_bundles.append(bundle_info[i][rider_index])
            to_dels.append(i)

            break

to_dels = set(to_dels)
bundle_info = [bundle_info[i] for i in range(len(bundle_info)) if i not in to_dels]
cost_info = [cost_info[i] for i in range(len(cost_info)) if i not in to_dels]

In [323]:
print(len(bundle_info))
for i in range(len(bundle_info)):
    print(i, f'{cost_info[i][0]}, {cost_info[i][1]}, {cost_info[i][2]}')

assert len(bundle_info) == len(cost_info) == 0

0


In [324]:
new_solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in new_all_bundles
]

## 결과 실현 가능성 확인

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

new_checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, new_solution)

print(new_checked_solution['avg_cost'])

3893.484
